# SciPy Practice — **Instructor Solutions**

This notebook provides complete worked solutions (code + explanations) for the companion student exercises covering one-sample proportion tests, confidence intervals for proportions, two-sample mean tests (Welch and pooled), and two-proportion z-tests — all using **SciPy** and **matplotlib** only.

**Conventions:**
- Random seeds fixed for reproducibility.
- Each chart in its own figure; no seaborn or custom color schemes.
- Where helpful, we show both *exact* and *approximate* methods.


In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

np.random.seed(42)  # reproducibility

def print_ci(label, low, high, conf=0.95):
    print(f"{label} {int(conf*100)}% CI: [{low:.4f}, {high:.4f}]")

def two_proportion_ztest(x1, n1, x2, n2, alternative="two-sided"):
    """Pooled two-proportion z-test for H0: p1 == p2.
    alternative in {"two-sided", "larger", "smaller"} refers to (p1 - p2).
    Returns (z, p).
    """
    p1_hat = x1 / n1
    p2_hat = x2 / n2
    p_pool = (x1 + x2) / (n1 + n2)
    se = np.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
    if se == 0:
        return np.nan, np.nan
    z = (p1_hat - p2_hat) / se
    if alternative == "two-sided":
        p = 2 * (1 - stats.norm.cdf(abs(z)))
    elif alternative == "larger":
        p = 1 - stats.norm.cdf(z)
    elif alternative == "smaller":
        p = stats.norm.cdf(z)
    else:
        raise ValueError("alternative must be 'two-sided', 'larger', or 'smaller'")
    return z, p


## 1) One-Sample Proportion — Test and Confidence Intervals (with Plot)

**Scenario:** Bernoulli process (e.g., email click-through). True `p=0.18`, sample `n=400`.


In [ ]:
# Data generation
p_true = 0.18
n = 400
data = stats.bernoulli.rvs(p_true, size=n)
x = data.sum()
phat = x / n
print(f"n={n}, x={x}, phat={phat:.4f}")


### H0: p = 0.20 vs H1: p ≠ 0.20 (Exact Binomial)
Use `stats.binomtest` and the exact Clopper–Pearson CI.

In [ ]:
res = stats.binomtest(k=x, n=n, p=0.20, alternative='two-sided')
print("Exact binomial test (p0=0.20): p-value =", f"{res.pvalue:.6f}")
ci_low95, ci_high95 = res.proportion_ci(confidence_level=0.95, method="exact")
print_ci("Exact (Clopper-Pearson)", ci_low95, ci_high95, conf=0.95)


### Normal Approximation (Wald) 95% CI
Shown for comparison/teaching purposes.

In [ ]:
z975 = stats.norm.ppf(0.975)
se = np.sqrt(phat*(1-phat)/n)
wald_low = phat - z975*se
wald_high = phat + z975*se
print_ci("Normal approx (Wald)", wald_low, wald_high)


### Plot: Proportion with 95% CI (Wald)

In [ ]:
plt.figure()
center = phat
err = (wald_high - wald_low)/2
plt.errorbar([0], [center], yerr=[err], fmt='o', capsize=8)
plt.xticks([0], ["Proportion"])
plt.title("Sample Proportion with 95% CI (Normal Approx)")
plt.ylabel("Proportion")
plt.ylim(0,1)
plt.show()


### **Exercise 1.1 Solution** — Change the null and re-test
Test `H0: p = 0.15` (two-sided). Also compute the **90% exact CI** and compare to the 95% CI.

In [ ]:
res15 = stats.binomtest(k=x, n=n, p=0.15, alternative='two-sided')
print("Exact binomial test (p0=0.15): p-value =", f"{res15.pvalue:.6f}")
ci_low90, ci_high90 = res15.proportion_ci(confidence_level=0.90, method="exact")
print_ci("Exact (Clopper-Pearson) 90%", ci_low90, ci_high90, conf=0.90)
print("\nObservation: The 90% CI is narrower than the 95% CI, as expected.")


## 2) Two Independent Means — Welch and Equal-Variance (with Plots)

**Scenario:** Time-on-page (seconds) for two landing pages A and B. We generate normal samples with slightly different means and SDs.

In [ ]:
# Generate data
nA, nB = 120, 140
muA, muB = 60, 65
sdA, sdB = 12, 16
A = np.random.normal(muA, sdA, size=nA)
B = np.random.normal(muB, sdB, size=nB)
print(f"A: n={nA}, mean={A.mean():.3f}, sd={A.std(ddof=1):.3f}")
print(f"B: n={nB}, mean={B.mean():.3f}, sd={B.std(ddof=1):.3f}")


### Welch's t-test (no equal-variance assumption)

In [ ]:
t_welch, p_welch = stats.ttest_ind(A, B, equal_var=False)
print(f"Welch t = {t_welch:.4f}, p-value = {p_welch:.6f}")


### Plots: Histograms (one figure per chart)

In [ ]:
plt.figure()
plt.hist(A, bins=20, alpha=0.7)
plt.title("Group A: Time on Page (seconds)")
plt.xlabel("Seconds"); plt.ylabel("Frequency")
plt.show()

plt.figure()
plt.hist(B, bins=20, alpha=0.7)
plt.title("Group B: Time on Page (seconds)")
plt.xlabel("Seconds"); plt.ylabel("Frequency")
plt.show()


### **Exercise 2.1 Solution** — Equal-variance t-test and 95% CI for μ_A − μ_B

We compute the pooled variance, the standard error of the difference, the t critical value, and the CI.

In [ ]:
# Equal-variance t-test
t_pooled, p_pooled = stats.ttest_ind(A, B, equal_var=True)
print(f"Pooled t-test: t = {t_pooled:.4f}, p-value = {p_pooled:.6f}")

# 95% CI for (mu_A - mu_B) under equal variances
meanA, meanB = A.mean(), B.mean()
s2A, s2B = A.var(ddof=1), B.var(ddof=1)
df = nA + nB - 2
Sp2 = ((nA-1)*s2A + (nB-1)*s2B) / df
SE_diff = np.sqrt(Sp2 * (1/nA + 1/nB))
tcrit = stats.t.ppf(0.975, df)
diff = meanA - meanB
ci_low = diff - tcrit*SE_diff
ci_high = diff + tcrit*SE_diff
print(f"Mean difference (A - B) = {diff:.3f}")
print_ci("Pooled t-based", ci_low, ci_high)


## 3) Two Proportions — Pooled z-test (with Plot)

**Scenario:** A/B test of a signup flow. We simulate counts for groups A and B and test `H0: p1 = p2`.

In [ ]:
n1, n2 = 600, 650
p1_true, p2_true = 0.22, 0.27
x1 = stats.binom.rvs(n1, p1_true)
x2 = stats.binom.rvs(n2, p2_true)
p1_hat, p2_hat = x1/n1, x2/n2
print(f"A: x1={x1}, n1={n1}, p1_hat={p1_hat:.4f}")
print(f"B: x2={x2}, n2={n2}, p2_hat={p2_hat:.4f}")
z2s, p2s = two_proportion_ztest(x1, n1, x2, n2, alternative="two-sided")
print(f"Two-sided z = {z2s:.4f}, p-value = {p2s:.6f}")


### Plot: Proportions with 95% Wald CIs

In [ ]:
z975 = stats.norm.ppf(0.975)
se1 = np.sqrt(p1_hat*(1-p1_hat)/n1)
se2 = np.sqrt(p2_hat*(1-p2_hat)/n2)
ci1 = (p1_hat - z975*se1, p1_hat + z975*se1)
ci2 = (p2_hat - z975*se2, p2_hat + z975*se2)

plt.figure()
plt.errorbar([0,1], [p1_hat, p2_hat], yerr=[(ci1[1]-ci1[0])/2, (ci2[1]-ci2[0])/2], fmt='o', capsize=8)
plt.xticks([0,1], ["Group A", "Group B"])
plt.title("Sample Proportions with 95% Wald CIs")
plt.ylabel("Proportion")
plt.ylim(0,1)
plt.show()


### **Exercise 3.1 Solution** — One-sided alternative `H1: p2 > p1`

Because the helper treats the alternative with respect to `(p1 - p2)`, the test `H1: p2 > p1` equivalently becomes `H1: (p1 - p2) < 0`, which corresponds to `alternative='smaller'` when we call `two_proportion_ztest(x1, n1, x2, n2, alternative='smaller')`.


In [ ]:
z1s, p1s = two_proportion_ztest(x1, n1, x2, n2, alternative='smaller')
print(f"One-sided test H1: p2>p1  => z = {z1s:.4f}, one-sided p-value = {p1s:.6f}")
if p1s < 0.05:
    print("Reject H0 at alpha = 0.05: evidence that p2 > p1.")
else:
    print("Fail to reject H0 at alpha = 0.05: insufficient evidence that p2 > p1.")


## 4) Practice — Instructor Exemplars

Below are example solutions for the open-ended practice prompts. Feel free to change parameters to explore behavior.


### 4.1 Proportions — Custom Example

In [ ]:
# Parameters (edit to explore)
p_custom = 0.30
n_custom = 250
data_c = stats.bernoulli.rvs(p_custom, size=n_custom)
x_c = data_c.sum()
phat_c = x_c / n_custom
print(f"Custom: n={n_custom}, x={x_c}, phat={phat_c:.4f}")

# Test H0: p = 0.25 (two-sided and one-sided 'greater')
res_ts = stats.binomtest(k=x_c, n=n_custom, p=0.25, alternative='two-sided')
res_gr = stats.binomtest(k=x_c, n=n_custom, p=0.25, alternative='greater')
print("Two-sided p-value:", f"{res_ts.pvalue:.6f}")
print("One-sided (p>0.25) p-value:", f"{res_gr.pvalue:.6f}")
ci90 = res_ts.proportion_ci(confidence_level=0.90, method='exact')
ci95 = res_ts.proportion_ci(confidence_level=0.95, method='exact')
ci99 = res_ts.proportion_ci(confidence_level=0.99, method='exact')
print_ci("Exact 90%", ci90.low, ci90.high, 0.90)
print_ci("Exact 95%", ci95.low, ci95.high, 0.95)
print_ci("Exact 99%", ci99.low, ci99.high, 0.99)

# Plot point estimate with 95% Wald CI
z975 = stats.norm.ppf(0.975)
se_c = np.sqrt(phat_c*(1-phat_c)/n_custom)
wald_low_c = phat_c - z975*se_c
wald_high_c = phat_c + z975*se_c
plt.figure()
plt.errorbar([0], [phat_c], yerr=[(wald_high_c - wald_low_c)/2], fmt='o', capsize=8)
plt.xticks([0], ["Custom p-hat"])
plt.title("Custom Proportion with 95% Wald CI")
plt.ylabel("Proportion")
plt.ylim(0,1)
plt.show()


### 4.2 Two Means — Custom Example

In [ ]:
# Parameters (edit to explore)
n1m, n2m = 80, 110
mu1m, mu2m = 50, 55
sd1m, sd2m = 10, 14
G1 = np.random.normal(mu1m, sd1m, size=n1m)
G2 = np.random.normal(mu2m, sd2m, size=n2m)
print(f"G1: n={n1m}, mean={G1.mean():.2f}, sd={G1.std(ddof=1):.2f}")
print(f"G2: n={n2m}, mean={G2.mean():.2f}, sd={G2.std(ddof=1):.2f}")

# Welch and pooled tests
t_w, p_w = stats.ttest_ind(G1, G2, equal_var=False)
t_p, p_p = stats.ttest_ind(G1, G2, equal_var=True)
print(f"Welch: t={t_w:.4f}, p={p_w:.6f}")
print(f"Pooled: t={t_p:.4f}, p={p_p:.6f}")

# 95% pooled CI for (mu1 - mu2)
s2_1, s2_2 = G1.var(ddof=1), G2.var(ddof=1)
dfm = n1m + n2m - 2
Sp2m = ((n1m-1)*s2_1 + (n2m-1)*s2_2) / dfm
SEm = np.sqrt(Sp2m*(1/n1m + 1/n2m))
tcritm = stats.t.ppf(0.975, dfm)
diffm = G1.mean() - G2.mean()
ciL, ciH = diffm - tcritm*SEm, diffm + tcritm*SEm
print_ci("Pooled t-based 95%", ciL, ciH)

# Plots
plt.figure()
plt.hist(G1, bins=18, alpha=0.7)
plt.title("G1 Distribution")
plt.xlabel("Value"); plt.ylabel("Frequency")
plt.show()

plt.figure()
plt.hist(G2, bins=18, alpha=0.7)
plt.title("G2 Distribution")
plt.xlabel("Value"); plt.ylabel("Frequency")
plt.show()


### 4.3 Two Proportions — Custom Example

In [ ]:
# Parameters (edit to explore)
n1p, n2p = 500, 520
pp1, pp2 = 0.18, 0.22
xx1 = stats.binom.rvs(n1p, pp1)
xx2 = stats.binom.rvs(n2p, pp2)
ph1, ph2 = xx1/n1p, xx2/n2p
print(f"xx1={xx1}, n1p={n1p}, ph1={ph1:.4f}")
print(f"xx2={xx2}, n2p={n2p}, ph2={ph2:.4f}")

# Two-sided and one-sided tests
z_ts, p_ts = two_proportion_ztest(xx1, n1p, xx2, n2p, alternative='two-sided')
z_gt, p_gt = two_proportion_ztest(xx1, n1p, xx2, n2p, alternative='smaller')  # H1: p2>p1
print(f"Two-sided: z={z_ts:.4f}, p={p_ts:.6f}")
print(f"One-sided (p2>p1): z={z_gt:.4f}, p={p_gt:.6f}")

# Plot Wald CIs
z975 = stats.norm.ppf(0.975)
se_1 = np.sqrt(ph1*(1-ph1)/n1p)
se_2 = np.sqrt(ph2*(1-ph2)/n2p)
ci_1 = (ph1 - z975*se_1, ph1 + z975*se_1)
ci_2 = (ph2 - z975*se_2, ph2 + z975*se_2)
plt.figure()
plt.errorbar([0,1], [ph1, ph2], yerr=[(ci_1[1]-ci_1[0])/2, (ci_2[1]-ci_2[0])/2], fmt='o', capsize=8)
plt.xticks([0,1], ["Group 1", "Group 2"])
plt.title("Two Proportions with 95% Wald CIs (Custom)")
plt.ylabel("Proportion")
plt.ylim(0,1)
plt.show()
